<a href="https://colab.research.google.com/github/simonseo/Computer-Science-Capstone-Lifting-Squares-to-a-Half-Space-a-Specific-Case-of-Lifting-Pseudo-disks/blob/master/cvxpy_approach_to_square_lifting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [CVXPY](http://www.cvxpy.org/tutorial/intro/index.html) Approach to the Square Liftability problem
We will try to find cases where squares are not liftable to half-spaces
## Issues
1. [DCPError: Problem does not follow DCP rules.](https://colab.research.google.com/drive/1f0xyAtKMKTFlUnlQOwIQE6SaJ80b0DJq?authuser=1#scrollTo=oVgKtqUDH9vW) Seems like `1` or `dummy_var**2` aren't accepted as proper objective functions. [DCP Rules](http://cvxr.com/cvx/doc/dcp.html)
1. Optimize point/square sampling by forcing some points to be inside the square
1. Use [Turtle](https://opentechschool.github.io/python-beginners/en/simple_drawing.html) to draw squares and points
1. Pretty print membership matrix

## Outline

1. Randomly sample m points in 2D (p_i)
1. Randomly sample 2n points that make n squares (S_j)
1. Save flags 
  ```python
   membership[i][j] = 1 if P_i.isinside(S_j) else -1
  ```

1. Create 3m variables representing 3D points (x_i,y_i,z_i, 1)
  ```python
  Q = cvx.Variable(m, 3)
  ```
1. Create 3n variables representing 3D half-spaces (a_i, b_i, c_i, 1): 
  ```python
   H = cvx.Variable(n, 3)
  ```
1. Create constraints so that if `P_i` is inside `S_j`, `Q_i` is on one side of `H_j`.  (Assume `Q_i = (x,y,z,1)`,`H_j = (a,b,c,1)` but leave out the 1 to save space):

  ```python
  for all i, j:
       constraints.append( (Q@H.T+1)[i][j]*membership[i][j] > 0 )
  ```
1. Set objective function as anything
1. Solve




In [0]:
!pip install cvxpy
!pip install numpy

    100% |████████████████████████████████| 901kB 19.6MB/s 
    100% |████████████████████████████████| 153kB 25.3MB/s 
    100% |████████████████████████████████| 153kB 31.3MB/s 
    100% |████████████████████████████████| 143kB 33.4MB/s 
    100% |████████████████████████████████| 1.4MB 12.1MB/s 
  Running setup.py bdist_wheel for cvxpy ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/8b/af/aa/46570716431521ee92085f317c33b2f427e27f08fe4a8a738a
  Running setup.py bdist_wheel for scs ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/ff/f0/aa/530ccd478d7d9900b4e9ef5bc5a39e895ce110bed3d3ac653e
  Running setup.py bdist_wheel for multiprocess ... - \ | / done
  Stored in directo

In [0]:
import cvxpy as cvx
import numpy as np
import random

#############
# Constants #
#############
BIG_NUMBER = 999999
SMALL_NUMBER = 1/BIG_NUMBER
# SMALL_NUMBER = cvx.Variable() # Tried this to see if we can minimize SMALL_NUMBER**2

GRID_SIZE = 1 # grid from which points will be sampled
m = 50 # number of points
n = 20 # number of squares


#########################################
# This are redefined in following cells #
#########################################

P = points_2d = [] # M list of Point Objects
S = squares = [] # N list of Square Objects

membership = [] # saves flag of whether point i is in square j

Q = var_points_3d = [] # M x 3 matrix
H = var_half_spaces = [] # N x 3 matrix



In [0]:
class Point():
  def __init__(self, x, y):
    self.x = x
    self.y = y
  
  def __repr__(self):
    return "({},{})".format(round(self.x, 2), round(self.y, 2))
    
  def isinside(self, square):
    s = square
    return (abs(s.center.x - self.x) <= s.radius) and (abs(s.center.y - self.y) <= s.radius)
    
    
    
class Square():
  def __init__(self, center, radius):
    self.center = center
    self.radius = radius
    
  def __repr__(self):
    return "[{},{}]x[{},{}]".format(round(self.center.x - self.radius, 2), round(self.center.x + self.radius, 2), 
                                    round(self.center.y - self.radius, 2), round(self.center.y + self.radius, 2))

In [0]:
def sample_points(m):
  # Randomly sample m points
  P = points_2d = [] # M list of Point
  for i in range(m):
    x = random.uniform(0, GRID_SIZE)
    y = random.uniform(0, GRID_SIZE)
    P.append(Point(x,y))
  return P



def sample_squares(n):
  # Randomly create n squares
  S = squares = [] # N list of Square
  for j in range(n):
    center_x = random.uniform(0, GRID_SIZE)
    center_y = random.uniform(0, GRID_SIZE)
    center = Point(center_x, center_y)
    radius = random.uniform(0, min(center_x, GRID_SIZE-center_x, center_y, GRID_SIZE-center_y))
    S.append(Square(center, radius))
  return S



def get_membership(P, S):
  # Membership saves the flag p_i.isinside(s_j)
  membership = []
  for i in range(m):
    tmp = []
    for j in range(n):
      tmp.append(1 if P[i].isinside(S[j]) else -1)
    membership.append(tmp)
  return np.array(membership)


P = sample_points(m)
S = sample_squares(n)
membership = get_membership(P, S)

In [0]:
#@title Default title text
# Create 3m variables representing 3D points ((x_i,y_i,z_i), 1): P[i][k] = cvx.Variable()
# Create 3n variables representing 3D half-spaces ((a_i, b_i, c_i), 1) H[j][k] = cvx.Variable()


def create_constraints(m, n, membership):
  Q = var_points_3d = cvx.Variable(shape=(m,3)) # M x 3 matrix of 3D points
  H = var_half_spaces = cvx.Variable(shape=(n,3)) # N x 3 matrix of 3D Half-spaces
  constraints = []
  
#   dot_product = Q@H.T+1
#   print(dot_product, dot_product[0][0])
  for i in range(m):
    for j in range(n):
      # dot(Qi,Hj) > 0 if point i inside square j, < 0 if outside
      # membership[i][j] is either 1 or -1
      # CVXPY does not support strict equality. Hence the use of SMALL_NUMBER.
      # CVXPY does not support Hadamard Product (elementwise product). Hence the use of a for-loop
#       constraints.append(dot_product[i][j]*membership[i][j] >= SMALL_NUMBER)
      pass
  a = cvx.Variable()
  b = cvx.Variable()
  c = cvx.Variable()
  x = cvx.Variable()
  y = cvx.Variable()
  z = cvx.Variable()
  
  constraints.append(a*x + b*y + c*z + 1 >= SMALL_NUMBER)

#   constraints.append(dot_product[0][0]*membership[0][0] <= SMALL_NUMBER)
  return constraints, Q, H




constraints, Q, H = create_constraints(m, n, membership)


In [0]:
obj = cvx.Minimize(1) # DCPError: Problem does not follow DCP rules. http://cvxr.com/cvx/doc/dcp.html

# fake_var = cvx.Variable()
# obj = cvx.Minimize(fake_var**2) # DCPError: Problem does not follow DCP rules.

# obj = cvx.Minimize(SMALL_NUMBER**2) # (if we set SMALL_NUMBER as a variable) DCPError: Problem does not follow DCP rules.

prob = cvx.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", H.value)


DCPError: ignored

In [0]:
prob.constraints

[NonPos(Expression(UNKNOWN, UNKNOWN, ()))]